In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/47_BERT_text_classification/spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [5]:
df_spam = df[df['Category']== 'spam']
df_spam.shape

(747, 2)

In [7]:
df_ham = df[df['Category']== 'ham']
df_ham.shape

(4825, 2)

In [8]:
df_ham_dowmSample = df_ham.sample(df_spam.shape[0])
df_ham_dowmSample.shape

(747, 2)

In [9]:
df_balanced = pd.concat([df_spam, df_ham_dowmSample], axis = 0)

In [10]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify=df_balanced['spam'])

In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [13]:
def get_sentence_embeddings(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeddings([
    "500$ discount. hurry up",
    "bhavin, are you up for a volleyball game tomorrow?"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84343696, -0.5132686 , -0.8885009 , ..., -0.7475339 ,
        -0.7530703 ,  0.9196097 ],
       [-0.8205707 , -0.5208476 , -0.95239025, ..., -0.88035464,
        -0.7052784 ,  0.82525545]], dtype=float32)>

In [15]:
e = get_sentence_embeddings([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]], [e[3]])

array([[0.8470749]], dtype=float32)

In [21]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [22]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer= 'adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [26]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 21s 590ms/step - loss: 0.3863 - accuracy: 0.8875 - precision: 0.8848 - recall: 0.8911
Epoch 2/10
35/35 [==============================] - 20s 565ms/step - loss: 0.3602 - accuracy: 0.8839 - precision: 0.8656 - recall: 0.9089
Epoch 3/10
35/35 [==============================] - 20s 581ms/step - loss: 0.3377 - accuracy: 0.8991 - precision: 0.8914 - recall: 0.9089
Epoch 4/10
35/35 [==============================] - 21s 595ms/step - loss: 0.3170 - accuracy: 0.9018 - precision: 0.8866 - recall: 0.9214
Epoch 5/10
35/35 [==============================] - 21s 594ms/step - loss: 0.3089 - accuracy: 0.9027 - precision: 0.8908 - recall: 0.9179
Epoch 6/10
35/35 [==============================] - 21s 597ms/step - loss: 0.2901 - accuracy: 0.9179 - precision: 0.9048 - recall: 0.9339
Epoch 7/10
35/35 [==============================] - 21s 586ms/step - loss: 0.2817 - accuracy: 0.9170 - precision: 0.9104 - recall: 0.9250
Epoch 8/10
35/35 [================

In [25]:
model.predict(["bhavin, are you up for a volleyball game tomorrow?"])

1/1 [==============================] - 4s 4s/step


array([[0.404041]], dtype=float32)

In [27]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 11s 605ms/step - loss: 0.2335 - accuracy: 0.9332 - precision: 0.9219 - recall: 0.9465


[0.23349808156490326, 0.9331550598144531, 0.921875, 0.9465240836143494]

In [28]:

y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

12/12 [==============================] - 10s 551ms/step


In [29]:


import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,

In [30]:

reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]

In [31]:
model.predict(reviews)

1/1 [==============================] - 0s 223ms/step


array([[0.8141546 ],
       [0.86456966],
       [0.7938542 ],
       [0.17210448],
       [0.06985648]], dtype=float32)

In [32]:
model.save(fr"D:\Models\Email Classification\1")

INFO:tensorflow:Assets written to: D:\Models\Email Classification\1\assets


INFO:tensorflow:Assets written to: D:\Models\Email Classification\1\assets
